In [ ]:
from dataset import Dataset
from classificadores import KNN, DMC, PerceptronSimples

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from typing import Union

In [ ]:
# Conjunto de dados Dermatologia =================================================
dataset = Dataset.from_file(
    filepath = r"datasets\dermatology.data",
    label_column = -1,
    column_names = ["erythema", "scaling", "definite-borders", "itching", "koebner" "phenomenon", "polygonal papules", "follicular papules", "oral-mucosal involvement", "knee elbow involvement", "scalp involvement", "family history", "melanin incontinence", "eosinophils in the infiltrate", "pnl infiltrate", "fibrosis of the papillary dermis", "exocytosis", "acanthosis", "hyperkeratosis", "parakeratosis", "clubbing of the rete ridges", "elongation of the rete ridges", "thinning of the suprapapillary epidermis", "spongiform pustule", "munro microabcess", "focal hypergranulosis", "disappearance of the granular layer", "vacuolisation and damage of the basal layer", "spongiosis", "saw-tooth appearance of retes", "follicular horn plug", "perifollicular parakeratosis", "inflammatory monoluclear infiltrate", "band-like infiltrate", "age", "class"]
).normalize()

print(dataset)

In [ ]:
# Conjunto de dados Vinho =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\wine.data", 
    label_column = 0, 
    column_names = ["class", "Alcohol", "Malicacid", "Ash", "Alcalinity of ash", "Magnesium", "Total phenols", "Flavanoids", "Nonflavanoid phenols","Proanthocyanins", "Color intensity", "Hue", "0D280 0D315 of diluted wines", "Proline"]
).normalize().move_label_to_end()

print(dataset)

In [ ]:
# Conjunto de dados Iris =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\iris.data", 
    label_column = -1, 
    column_names = ["sepal length", "sepal width", "petal length", "petal width", "class"]
).ensure_numeric_labels().normalize()

print(dataset)
dataset._label_index_to_name

In [ ]:
# Conjunto de dados Coluna Vertebral =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\column_3C.dat", 
    label_column = -1,
    delimiter = " ",  
    column_names = ["pelvic incidence", "pelvic tilt", "lumbar lordosis angle", "sacral slope", "pelvic radius", "degree spondylolisthesis", "class"]
).ensure_numeric_labels().normalize()

print(dataset)
dataset._label_index_to_name

In [ ]:
dataset.data

In [ ]:
dataset.determination_matrix()

In [ ]:
dataset.vectorize_labels()
dataset.label_encodings

In [ ]:
# Separa o conjunto de dados em treinamento e teste
train_dataset, test_dataset = dataset.split()
train_dataset : Dataset 
test_dataset : Dataset

In [ ]:
q = 6                                   # Número de neurônios ocultos
m = train_dataset.class_count           # Número de neurônios de saída
p = train_dataset.features_count + 1    # Número de entradas da rede
eta = 0.1                               # Taxa de aprendizado

# Função de ativação usada
activation = lambda x: np.tanh(x)
ddx_activation = lambda x: 1 - activation(x) ** 2

# Inicializa o vetor de pesos dos neurônios ocultos
W = np.random.normal( size = (q, p) )

# Inicializa o vetor de pesos dos neurônios de saída
M = np.random.normal( size = (m, q+1) )

# Número de épocas de treinamento
max_epocas = 5000

# Percorre um número qualquer de épocas
for epoca in range( max_epocas ):
    total_erros = 0

    # Para cada época, embaralha o conjunto de treinamento
    shuffled_dataset = train_dataset.shuffle()

    # Percorre os exemplos do conjunto de treinamento
    for index, *features, classe in shuffled_dataset:
        # Vetor que representa a classe 
        real_output = train_dataset.encode_label( classe )

        # Monta o vetor de entrada
        X_bias = np.r_[features, -1]

        # Sentido direto - cálculo da ativação e a saída de cada camada
        U = W @ X_bias          # Ativação de cada neurônio oculto
        Y = activation( U )     # Saída dos neurônios ocultos

        Z = np.r_[ Y, -1 ]      # Prepara as entradas para os neurônios de saída

        A = M @ Z               # Ativação dos neurônio de saída
        O = activation ( A )    # Saída da camada de saída

        # Sentido inverso - atualização dos pesos das camadas
        err = real_output - O 

        # Verifica se houve erros
        if np.any(err != 0):
            # atualiza os pesos da camada de saída
            delta_output = err * ddx_activation( A )                        # (m x 1)
            delta_weights_out = eta * np.outer( delta_output, Z )
            M = M + delta_weights_out

            # calcula os erros retropagados para a camada oculta oculto
            output_weights_no_bias_T = M[:, :-1].T                          # (q x m)
            backpropagated_error = output_weights_no_bias_T @ delta_output  # (q x 1)

            # atualiza os pesos da camada oculta
            delta_hidden = backpropagated_error * ddx_activation(U)
            delta_weights_hidden = eta * np.outer( delta_hidden, X_bias )
            W = W + delta_weights_hidden

        # Monta um vetor de predição baseado no argmax da saída da rede
        predicted_class = -np.ones_like(O)
        predicted_class[ np.argmax(O) ] = +1

        if np.argmax(O) != np.argmax(real_output):
            total_erros += 1

    if (epoca% round(max_epocas*0.05)) == 0:
        print(f"Época {epoca}: {total_erros} erros. - saída {predicted_class} [{real_output}]")
        

In [ ]:
PS = PerceptronSimples( train_dataset )
PS.train( 500 )

for index, *point_test, classe in test_dataset:
    classe_prevista = PS.predict( point_test )
    print(f"{index}] Previu {classe_prevista} e era {classe} [{classe_prevista == classe}]")

train_dataset._centroids